In [ ]:
from kgfw.extract.doc_handle import BaseDocxHandle

document = BaseDocxHandle('Function Description XCS.docx')
data = document.get_docx_structure()
# catalog = document.get_catalog()
# document.recovery_docx()

In [ ]:
from kg_extract.doc_handle import BasePptxHandle
document = BasePptxHandle('test.pptx')
data = document.get_pptx_structure()

In [ ]:
import pandas as pd

data = pd.read_excel('dublin2229_GPIO_Pinmux_PA1_upadte.xlsx')

In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
# es_ctrl = Elasticsearch([{'host':'localhost','port':9200}])
es_ctrl = Elasticsearch([{'host':'10.166.152.49','port':9200}])

In [ ]:
import re
import json
data = es_ctrl.get(index='trouble-shooting-task', id='9jZFRXYBvGheiu9GCm8o')
# re.sub('\"topic\": (.*?)', '',json.dumps(data))

In [ ]:
update_data = {'doc':{'username': 'executor',
     'password': '123456',
     'authority': 'normal',
     'groups': [
        {
             'project': 'Common',
             'role': 'visitor'
         },
         {
             'project': 'TroubleShooting',
             'role': 'executor'
         }
     ]}}

es_ctrl.update(index='admin', id='jwbx83UBOPmBSJMOKjDC', body=update_data)

In [ ]:
pages = [{
    'project': 'Common',
    'pages':[{'name':'pinmux','role':['visitor'],'href':'#/pinmux'},
             {'name':'feedback','role':['visitor'],'href':'#/feedback'}
            ]
    },
    {
    'project': 'Babel',
    'pages':[{'name':'symbols','role':['visitor'],'href':'#/symbols'},
             {'name':'viewer','role':['visitor'],'href':'#/viewer'}
            ]       
    },
    {
    'project': 'TroubleShooting',
    'pages':[{'name':'edit','role':['administrator','executor','visitor'],'href':'#/trouble_shooting_template_edit'},
            {'name':'task','role':['administrator','executor','visitor'],'href':'#/trouble_shooting_task'}
            ]       
    }
]

for elm in pages:
    _ = es_ctrl.index(index='pages', body=elm)

In [4]:
es_ctrl.search(index='admin')

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [3]:
doc = {
    "mappings":{
        "properties":{
            "username":{
                "type":"keyword"
            },
            "cookie":{
                "type":"keyword"
            }
        }
    }
}
es_ctrl.indices.create(index='admin',body =doc)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'admin'}

In [2]:
es_ctrl.indices.delete(index='admin')

{'acknowledged': True}

In [ ]:
doc = {
            "query": {
                "bool": {
                    "must": [
                        {
                            "term": {
                                "template_id": "ojvTSnYBChHa9MqEsktD"
                            }
                        }
                    ]
                }
            }
}

es_ctrl.search(index='trouble-shooting-task', body=doc)

In [ ]:
es_ctrl.delete(index='trouble-shooting-checklist-comments', id='thKuK3YBK326mpxBfh9q')

In [ ]:
data = es_ctrl.search(index='trouble-shooting-task')['hits']['hits'][0]['_source']
data

In [ ]:
from io import BytesIO
import pandas as pd

def mind_export_to_csv(node_data, node=None):
    temp = []
    node = node if node else ''
    if type(node_data) == list:
        for elm in node_data:
            if elm.__contains__('children'):
                temp.extend(mind_export_to_csv(elm['children'], node+'/'+elm['topic']))
            else:
                temp.append([elm['topic'], node, elm['Status'], elm['Executor']])
    else:
        if node_data.__contains__('children'):
            temp.extend(mind_export_to_csv(node_data['children'], node+'/'+node_data['topic']))
        else:
            temp.append([node_data['topic'], node, node_data['Status'], node_data['Executor']])
    return temp

res = es_ctrl.get(index='trouble-shooting-task', id='6Tu7S3YBChHa9MqEu0za')
data = res['_source']['nodeData']['children']
temp = mind_export_to_csv(data)
temp = pd.DataFrame(temp, columns=['Tasks', 'Responsible', 'Achieved', 'Executor'])

# excel_file = BytesIO()
xlwriter = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')
temp.to_excel(xlwriter, res['_source']['TemplateName'])
# xlwriter.save()
# excel_file.seek(0)

In [ ]:
workbook = xlwriter.book
worksheet = xlwriter.sheets[res['_source']['TemplateName']]

format_l = workbook.add_format()
format_l.set_pattern(1)
format_l.set_bg_color('gray')
worksheet.write('B1:D2', 'Ray', format_l)
xlwriter.save()

In [ ]:
worksheet.write?

In [ ]:
#coding: utf-8
import xlsxwriter
 
# 创建工作簿
file_name = "first_book.xlsx"
workbook = xlsxwriter.Workbook(file_name)
 
# 创建工作表
worksheet = workbook.add_worksheet('sheet1')
 
# 写单元格
worksheet.write(0, 0, 'id')
worksheet.write(0,1, 'name')
worksheet.write(0,2, 'class')
worksheet.write(0,3, 'data')
 
# 写行
worksheet.write_row(1, 0, [1, 2, 3])
 
# 写列,其中列D需要大写
worksheet.write_column('D2', ['a', 'b', 'c'])
 
# 关闭工作簿
workbook.close()

In [ ]:
import requests


response = requests.get("http://127.0.0.1:5010/get/")

In [ ]:
response.content

In [ ]:
import pandas as pd

data = pd.read_csv('test.csv', encoding='gbk')
data.iloc[:,[0]].values

In [ ]:
import base64

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = 'administrator'.encode('utf-8')
key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)

file_out = open("encrypted.bin", "wb")
[ file_out.write(x) for x in (cipher.nonce, tag, ciphertext) ]
file_out.close()

In [ ]:
file_in = open("encrypted.bin", "rb")
nonce, tag, ciphertext = [ file_in.read(x) for x in (16, 16, -1) ]

# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(ciphertext, tag)
data

In [ ]:
from Crypto.PublicKey import RSA

key = RSA.generate(1024)
private_key = key.export_key()
file_out = open("private.pem", "wb")
file_out.write(private_key)
file_out.close()

public_key = key.publickey().export_key()
file_out = open("receiver.pem", "wb")
file_out.write(public_key)
file_out.close()

In [ ]:
cipher.block_size

In [ ]:
import requests

In [ ]:
import gzipimport base64
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Cipher import PKCS1_v1_5

ciphertext = 'mzlvr1MNY6y2k3TZvh3DdoGyrNu7xy2AfUXSTaT+Gma+ceZo2Cn4Nixep2dIkeyHW8RKj1JNgZr5eGn8ErgRtPzYx2+tvBCt1pXuZK80LGvfc3ajDbiwd1FkuJvnRcyPrayaD8jfG55EJv0RL/P2+bkyco3cNCAtR2A+Ys7/GMA='
private_key = RSA.import_key(open("private.pem").read())


# Decrypt the session key with the private RSA key
cipher_rsa = PKCS1_v1_5.new(private_key)
session_key = cipher_rsa.decrypt(base64.b64decode(ciphertext),"ERROR")

# # Decrypt the data with the AES session key
# cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
# data = cipher_aes.decrypt_and_verify(ciphertext, tag)
# print(data.decode("utf-8"))

In [ ]:
session_key.decode()

In [ ]:
a = "{\"isNewReport\":1,\"userId\":\"\",\"carNo\":\"甘-J2L199\",\"privacyOption\":1,\"longitude\":103.97679,\"latitude\":30.67841,\"distance\":3000,\"openId\":\"\",\"signType\":\"MD5\",\"timestamp\":1609574891332,\"nonce\":\"35053231609574891332\",\"sign\":\"DA8FEA8741ACBA2F7B7B8A2BDE994F8D\"}"
b = "{\"isNewReport\":1,\"userId\":\"c36b86825b004d648bfbbbc1b58a675e\",\"carNo\":\"川-C3D682\",\"privacyOption\":1,\"longitude\":0,\"latitude\":0,\"distance\":3000,\"openId\":\"\",\"signType\":\"MD5\",\"timestamp\":1609739888281,\"nonce\":\"75341421609739888281\",\"sign\":\"48DF5B8598ADEC7BE0D1B90D268C227E\"}"

In [ ]:
import gzip
import json

with open('IWD.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)

In [ ]:
len(gzip.compress(json.dumps(json_data).encode()))

In [ ]:
len(json.dumps(json_data))

In [1]:
import requests

url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_api/web/GetListUsingPath(DecodedUrl=@a1)/RenderListDataAsStream?@a1='/sites/PDURadioChengduFirmware/Shared Documents'&View=1d04f153-2aa3-4553-b6e2-14b66655e6d4&TryNewExperienceSingle=TRUE"
headers={
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    'Accept': "application/json;odata=verbose",
    'Accept-Language': "en-US,en;q=0.5",
    'Accept-Encoding': "gzip, deflate, br",
    'Content-Type': "application/json;odata=verbose",
    'Content-Length': "166",
    'X-SP-REQUESTRESOURCES': "NAVIGATIONSTRUCTURE, listUrl=%2Fsites%2FPDURadioChengduFirmware%2FShared%20Documents",
    'X-ServiceWorker-Strategy': "CacheFirst",
    'Origin': "https://ericsson.sharepoint.com",
    'Referer': "https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/Shared%20Documents/Forms/AllItems.aspx",
    'Connection': "keep-alive",
    'Cookie': "rtFa=WSBaz0JfgJmPAlBlLH9kCuaTD118ctqDxr0kXGnh8iwmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU2MjAxMzE5NzUzNjgwOSMyMjRBQTU5Ri1EMDRCLUIwMDAtQzk0Mi00QzVDRUQzQUNGMUYozAClMgzoD5Fb1dM6x/qPug79a5y4Vm4RVuDMFNgRBSJ+AmS6ksdUyyrfQTr1aWxmsYQ9dIyTWb4LxNO+lZy6z54d3i7rvJ2fPYGRXbmWUKyjyg54ts1B7NKL31q0OSLTrmnt6XdSKoSaUy2X1znQG9oVPpWNfMvmxUWOScpOluNYaHv+3iY14kvLGJKZPZ1jVbG4JDDI1T1VEzAFkaK2aSQHuEfBr5TUj7d0mm1u2D6Ak6hlH6k1hsuGmiM4MU8/n//yQhfjrB0Ujw4InleEjQSGX/QCb/KOUeTULRHP18+JVeprkQn0II/rcUvWx6iTz3qne4p8Y2MwXCjJhaiDfQAAAA==; FedAuth=77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI1NjIwMTI5NTAwMDAwMDAsMTMyNTIzODM2MDAwMDAwMDAwLDEzMjU2Mjg3NzE5NzM4MDMwOSwxMDEuMzMuMTI3LjExNSw2Niw5MmU4NGNlYi1mYmZkLTQ3YWItYmU1Mi0wODBjNmI4Nzk1M2YsLDRiOWYxYzUyLTdiYzktNGQyZC1iYzMxLTI4ZDBlYmU2OTNhOSwyMmEyODc5ZC02ODdlLTQ0N2YtODBiZC1mYmM4YTU0NDdiOWQsMjJhMjg3OWQtNjg3ZS00NDdmLTgwYmQtZmJjOGE1NDQ3YjlkLCwwLDEzMjU2MjA0OTE5NjkxMTk2NywxMzI1NjQ2MDUxOTY5MTE5NjcsLCxleUo0YlhOZlkyTWlPaUpiWENKRFVERmNJbDBpZlE9PSwyNjUwNDY3NzQzOTk5OTk5OTk5LDEzMjU2MjAxMzE4MDAwMDAwMCxlYzhmOTFhNS01NDY1LTQ2NmYtYjMwOS00NGQxZDNkMGUyODAsSUhuNWlPWDNMNXlhazRsRHpXMi9kY01GVmgyMGZmbzNxUlpYRkU4emZaZGxZdEJYUjBWTjV5OW80SFFRVDN5cDRuRk9HK3pJcVNwV0dmU09CZEI3Q2QrcWF5eUM0cU84WTB3Tm51cWYvNktRVC9nRzk4ZEN0enpGZHo2Tm5qSUY3Z2JLenZoK1paVFpVRVFEZUZ4UVBaZUxNcXA4cTVqQkZOOTRqZFErMXJLZElETHpzcFJzSmxEbnZYSStlOE5ZUzc5ZWxrYU9iSUp2bWp6cStSWVYwUnpJQ1E3OHZNam9FQU9CVzR4MnFLMVpmd29vVmZQbDBWMWtsRmVoSElPc1B0NlZnd2dmSVljcFNteVdGVHZiSXFoamxPUkZKT2xKbG9VREI3K0d5Y2I0UmNyUEpxOGZZSUxmNVBhR3YrZ041R2VhMlNuNWtUenE2SXJGQkZwRHZ3PT08L1NQPg==; CCSInfo=MS8zMC8yMDIxIDg6MTM6MzcgQU1Dq8P8FwjF7qoLaotIfXAD6pzzfAC8Igyur4WzdR48fhkTNrX4T+U+3N3OGnqnnUrgXhcg4TBb1y4oj4ElQoVwUSiMNMYySW9/trH+ncjNMtm1KXp8MzcwHnTeA2pyw0mwz11zlQAr0cuoo0sfcVUiPj2hFTZIVN90z2pDmmAL62MFZEANBNPMo0/ul6PgfNsMo4JMggLqUR4JWdV4ePmD11m0t+U2o7G5r/xeXyuYLLQvNdsl2BWtCCBeB3DbMUMJz+yOzqmXFnInWBUwy6dyAgZXupBc2D+ukeXWcP/6SKyKiP47iZHOsQRx5dV4KSNa9sJXfUMs4fK1eZH5+lnrFAAAAA==; isso=dHJ1ZSwyODUwMCwyODgwMCwxNjExNzU2Mjk1NTI3LDE2MTE3NTY1OTU1Mjc=; SPWorkLoadAttribution=Url=https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/Shared%20Documents/Forms/AllItems.aspx&AppTitle=ModernDoclibListPage",
    'TE': "Trailers"
}
payload={"parameters":{"__metadata":{"type":"SP.RenderListDataParameters"},"RenderOptions":5707527,"AllowMultipleValueFilterForTaxonomyFields":True,"AddRequiredFields":True}}
cookies={
    'CCSInfo': "MS8zMC8yMDIxIDg6MTM6MzcgQU1Dq8P8FwjF7qoLaotIfXAD6pzzfAC8Igyur4WzdR48fhkTNrX4T+U+3N3OGnqnnUrgXhcg4TBb1y4oj4ElQoVwUSiMNMYySW9/trH+ncjNMtm1KXp8MzcwHnTeA2pyw0mwz11zlQAr0cuoo0sfcVUiPj2hFTZIVN90z2pDmmAL62MFZEANBNPMo0/ul6PgfNsMo4JMggLqUR4JWdV4ePmD11m0t+U2o7G5r/xeXyuYLLQvNdsl2BWtCCBeB3DbMUMJz+yOzqmXFnInWBUwy6dyAgZXupBc2D+ukeXWcP/6SKyKiP47iZHOsQRx5dV4KSNa9sJXfUMs4fK1eZH5+lnrFAAAAA==",
    'FedAuth': "77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI1NjIwMTI5NTAwMDAwMDAsMTMyNTIzODM2MDAwMDAwMDAwLDEzMjU2Mjg3NzE5NzM4MDMwOSwxMDEuMzMuMTI3LjExNSw2Niw5MmU4NGNlYi1mYmZkLTQ3YWItYmU1Mi0wODBjNmI4Nzk1M2YsLDRiOWYxYzUyLTdiYzktNGQyZC1iYzMxLTI4ZDBlYmU2OTNhOSwyMmEyODc5ZC02ODdlLTQ0N2YtODBiZC1mYmM4YTU0NDdiOWQsMjJhMjg3OWQtNjg3ZS00NDdmLTgwYmQtZmJjOGE1NDQ3YjlkLCwwLDEzMjU2MjA0OTE5NjkxMTk2NywxMzI1NjQ…FhNS01NDY1LTQ2NmYtYjMwOS00NGQxZDNkMGUyODAsSUhuNWlPWDNMNXlhazRsRHpXMi9kY01GVmgyMGZmbzNxUlpYRkU4emZaZGxZdEJYUjBWTjV5OW80SFFRVDN5cDRuRk9HK3pJcVNwV0dmU09CZEI3Q2QrcWF5eUM0cU84WTB3Tm51cWYvNktRVC9nRzk4ZEN0enpGZHo2Tm5qSUY3Z2JLenZoK1paVFpVRVFEZUZ4UVBaZUxNcXA4cTVqQkZOOTRqZFErMXJLZElETHpzcFJzSmxEbnZYSStlOE5ZUzc5ZWxrYU9iSUp2bWp6cStSWVYwUnpJQ1E3OHZNam9FQU9CVzR4MnFLMVpmd29vVmZQbDBWMWtsRmVoSElPc1B0NlZnd2dmSVljcFNteVdGVHZiSXFoamxPUkZKT2xKbG9VREI3K0d5Y2I0UmNyUEpxOGZZSUxmNVBhR3YrZ041R2VhMlNuNWtUenE2SXJGQkZwRHZ3PT08L1NQPg==",
    'isso':"dHJ1ZSwyODUwMCwyODgwMCwxNjExNzU2Mjk1NTI3LDE2MTE3NTY1OTU1Mjc=",
    'rtFa':"WSBaz0JfgJmPAlBlLH9kCuaTD118ctqDxr0kXGnh8iwmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU2MjAxMzE5NzUzNjgwOSMyMjRBQTU5Ri1EMDRCLUIwMDAtQzk0Mi00QzVDRUQzQUNGMUYozAClMgzoD5Fb1dM6x/qPug79a5y4Vm4RVuDMFNgRBSJ+AmS6ksdUyyrfQTr1aWxmsYQ9dIyTWb4LxNO+lZy6z54d3i7rvJ2fPYGRXbmWUKyjyg54ts1B7NKL31q0OSLTrmnt6XdSKoSaUy2X1znQG9oVPpWNfMvmxUWOScpOluNYaHv+3iY14kvLGJKZPZ1jVbG4JDDI1T1VEzAFkaK2aSQHuEfBr5TUj7d0mm1u2D6Ak6hlH6k1hsuGmiM4MU8/n//yQhfjrB0Ujw4InleEjQSGX/QCb/KOUeTULRHP18+JVeprkQn0II/rcUvWx6iTz3qne4p8Y2MwXCjJhaiDfQAAAA==",
    'SPWorkLoadAttribution':"Url=https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/Shared Documents/Forms/AllItems.aspx&AppTitle=ModernDoclibListPage"
}

In [4]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/Shared%20Documents/Forms/AllItems.aspx"
headers={
    'Host': "ericsson.sharepoint.com",
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'Accept-Language': "en-US,en;q=0.5",
    'Accept-Encoding': "gzip, deflate, br",
    'Referer': "https://login.microsoftonline.com/",
    'Upgrade-Insecure-Requests': "1",
    'Connection': "keep-alive",
    'Cookie': "rtFa=WSBaz0JfgJmPAlBlLH9kCuaTD118ctqDxr0kXGnh8iwmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU2MjAxMzE5NzUzNjgwOSMyMjRBQTU5Ri1EMDRCLUIwMDAtQzk0Mi00QzVDRUQzQUNGMUYozAClMgzoD5Fb1dM6x/qPug79a5y4Vm4RVuDMFNgRBSJ+AmS6ksdUyyrfQTr1aWxmsYQ9dIyTWb4LxNO+lZy6z54d3i7rvJ2fPYGRXbmWUKyjyg54ts1B7NKL31q0OSLTrmnt6XdSKoSaUy2X1znQG9oVPpWNfMvmxUWOScpOluNYaHv+3iY14kvLGJKZPZ1jVbG4JDDI1T1VEzAFkaK2aSQHuEfBr5TUj7d0mm1u2D6Ak6hlH6k1hsuGmiM4MU8/n//yQhfjrB0Ujw4InleEjQSGX/QCb/KOUeTULRHP18+JVeprkQn0II/rcUvWx6iTz3qne4p8Y2MwXCjJhaiDfQAAAA==; FedAuth=77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI1NjIwMTI5NTAwMDAwMDAsMTMyNTIzODM2MDAwMDAwMDAwLDEzMjU2Mjg3NzE5NzM4MDMwOSwxMDEuMzMuMTI3LjExNSw2Niw5MmU4NGNlYi1mYmZkLTQ3YWItYmU1Mi0wODBjNmI4Nzk1M2YsLDRiOWYxYzUyLTdiYzktNGQyZC1iYzMxLTI4ZDBlYmU2OTNhOSwyMmEyODc5ZC02ODdlLTQ0N2YtODBiZC1mYmM4YTU0NDdiOWQsMjJhMjg3OWQtNjg3ZS00NDdmLTgwYmQtZmJjOGE1NDQ3YjlkLCwwLDEzMjU2MjA0OTE5NjkxMTk2NywxMzI1NjQ2MDUxOTY5MTE5NjcsLCxleUo0YlhOZlkyTWlPaUpiWENKRFVERmNJbDBpZlE9PSwyNjUwNDY3NzQzOTk5OTk5OTk5LDEzMjU2MjAxMzE4MDAwMDAwMCxlYzhmOTFhNS01NDY1LTQ2NmYtYjMwOS00NGQxZDNkMGUyODAsSUhuNWlPWDNMNXlhazRsRHpXMi9kY01GVmgyMGZmbzNxUlpYRkU4emZaZGxZdEJYUjBWTjV5OW80SFFRVDN5cDRuRk9HK3pJcVNwV0dmU09CZEI3Q2QrcWF5eUM0cU84WTB3Tm51cWYvNktRVC9nRzk4ZEN0enpGZHo2Tm5qSUY3Z2JLenZoK1paVFpVRVFEZUZ4UVBaZUxNcXA4cTVqQkZOOTRqZFErMXJLZElETHpzcFJzSmxEbnZYSStlOE5ZUzc5ZWxrYU9iSUp2bWp6cStSWVYwUnpJQ1E3OHZNam9FQU9CVzR4MnFLMVpmd29vVmZQbDBWMWtsRmVoSElPc1B0NlZnd2dmSVljcFNteVdGVHZiSXFoamxPUkZKT2xKbG9VREI3K0d5Y2I0UmNyUEpxOGZZSUxmNVBhR3YrZ041R2VhMlNuNWtUenE2SXJGQkZwRHZ3PT08L1NQPg==; CCSInfo=MS8zMC8yMDIxIDg6MTM6MzcgQU1Dq8P8FwjF7qoLaotIfXAD6pzzfAC8Igyur4WzdR48fhkTNrX4T+U+3N3OGnqnnUrgXhcg4TBb1y4oj4ElQoVwUSiMNMYySW9/trH+ncjNMtm1KXp8MzcwHnTeA2pyw0mwz11zlQAr0cuoo0sfcVUiPj2hFTZIVN90z2pDmmAL62MFZEANBNPMo0/ul6PgfNsMo4JMggLqUR4JWdV4ePmD11m0t+U2o7G5r/xeXyuYLLQvNdsl2BWtCCBeB3DbMUMJz+yOzqmXFnInWBUwy6dyAgZXupBc2D+ukeXWcP/6SKyKiP47iZHOsQRx5dV4KSNa9sJXfUMs4fK1eZH5+lnrFAAAAA==; isso=dHJ1ZSwyODUwMCwyODgwMCwxNjExNzU2MjkxNDk2LDE2MTE3NTY1OTE0OTY="
}
payload={"parameters":{"__metadata":{"type":"SP.RenderListDataParameters"},"RenderOptions":5707527,"AllowMultipleValueFilterForTaxonomyFields":True,"AddRequiredFields":True}}
cookies={
    'CCSInfo': "Mi8yNS8yMDIxIDQ6NTc6MDggQU0+y3+gRY7eP9M4Rb4QItWUrtKwBxMwjhrfkXUcLq4ds7aXMQLfA+StZVv47me9vAY/0asQp8DcgZig/TVrDIpag5+bMVUUlHTyoXxHHHzcTea4a474tY6vZzuqZKpvgmJGpluoP4g/4snVxnJV3tTTP1NWFO/7xdql0uwwAaT2WTf6kx2vQTUwEfv++MyCq7Ah9Ne98tVwdAfl5HWxE28ypmARQbFWNyaLrcAo+v/58omzPDhBHSGzNJBW078j/fqJLZevJBfrY1S9dRtMNFYg8gCYqwsmBeaXe+jU7pgNtn3/6nUlpbGsKcRlWzL9B/WfuJeXg+Y7pwfNRvEkfqt0FAAAAA==",
    'FedAuth': "77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI1ODQzNTkwMTAwMDAwMDAsMTMyNTIzODM2MDAwMDAwMDAwLDEzMjU4NTIyMzMwMjQ1OTU4MiwxMDEuMzMuMTI3LjExNSw2Niw5MmU4NGNlYi1mYmZkLTQ3YWItYmU1Mi0wODBjNmI4Nzk1M2YsLDNmYWUxYzM5LWEzOTktNDg0MC1iZmViLTc5NjgwMWJmNDczNCxmMzNhM2U0Ni1hNjgyLTRmMDEtODg1My02YjVkNDI0MTg0OTEsZjMzYTNlNDYtYTY4Mi00ZjAxLTg4NTMtNmI1ZDQyNDE4NDkxLCwwLDEzMjU4NDM5NTMwMDg5NzM1NiwxMzI1ODY5NTEzMDA4OTczNTYsLCxleUo0YlhOZlkyTWlPaUpiWENKRFVERmNJbDBpZlE9PSwyNjUwNDY3NzQzOTk5OTk5OTk5LDEzMjU4NDM1OTI5MDAwMDAwMCxlYzhmOTFhNS01NDY1LTQ2NmYtYjMwOS00NGQxZDNkMGUyODAsdVZ5emxkOGpuVzhhR3ljUTR2Z21RblZoL3UxWGxYTlV4WVZMcjZTaXZxd2pobGxLcmJMOWlvcDZTWGV6VkZTRHB5eVJBWDRGdGd5R3RraWhhMDZEaGozaVdzRWdnTTQrTmEyUDVWODVoNGNZQWo5QzRtU1F3RzNaZGpoMkZ5OHNUcVF1K3lhRFRYWkZKSXgwdHN2UFArMUthVHFiUTM5SDZKdXhRRWpVeWFURWtiYlpDT1M2M1M4dTdkMkYxYVNPZjhNaXdWaXIvVlF0QjJzWWpqWUtXUFRXckMwcmJ3UmlZYWFDMXR0b2dMZC9iNGdpa2RXRGpMT0h2bkdUcDVJaEhwNlUwd1h0R2l1dkdvazdCYVVPeEduNHkxZHJPU2VpdE1yWm5lSWc4ZnBxaUtCSWFVU2NXcGtiaksxZTM1dVZwdmJFUFNKYlEyVHRDbDA5STZSbHBRPT08L1NQPg==",
    'isso':"dHJ1ZSwyODUwMCwyODgwMCwxNjEzOTkwODc1NTYxLDE2MTM5OTExNzU1NjE=",
    'rtFa':"OO+bIOYOBvyQa/pn+TjR0ere2fxDh7GzDRPBQ1f0cEcmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU4NDM1OTMwMjYxNjEzNiMzOTlEQUQ5Ri1BMDdELUIwMDAtQ0M4Ny1BRTkwMzI2Q0EzNzIBxW0xUovBgycOQ3YMd/qCMlEO80n0nd9Vhw23l6YobmRMWApdLwRz+lip86ew2dlMnXl53/RvYsEBm/tTLzTk649jJ6MywZFHnWGVmhFAsa9LT3PdeRhwqeohzWF1j+XqMng4sYg76Ub0QHJhTx23ywr2jrAwikeNrtizTLPS0CM4/qiVsZAnBhMORqb7fSxLrK1M/+KD2t6m1uiW5b6wjt4iI1kAY7P+sjH+rzgxtoZFbCqStH5Duk1b6gapYs7iggbbaHGcUUrdiEBw90o8a2z0Ybxn3T0M+tpQAX7gD4k765uDlNFlO+TmmU36M98K0TI88Yh+hxrp2ptyAAIzfQAAAA==",
}

In [5]:
request = requests.session()
ret=request.get(url,headers=headers,cookies=cookies)
print(ret.cookies.get_dict())

{'isso': 'dHJ1ZSwyODUwMCwyODgwMCwxNjEzOTkxMDU4NDkyLDE2MTM5OTEzNTg0OTI=', 'rtFa': 'OO+bIOYOBvyQa/pn+TjR0ere2fxDh7GzDRPBQ1f0cEcmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU4NDM1OTMwMjYxNjEzNiMzOTlEQUQ5Ri1BMDdELUIwMDAtQ0M4Ny1BRTkwMzI2Q0EzNzIBxW0xUovBgycOQ3YMd/qCMlEO80n0nd9Vhw23l6YobmRMWApdLwRz+lip86ew2dlMnXl53/RvYsEBm/tTLzTk649jJ6MywZFHnWGVmhFAsa9LT3PdeRhwqeohzWF1j+XqMng4sYg76Ub0QHJhTx23ywr2jrAwikeNrtizTLPS0CM4/qiVsZAnBhMORqb7fSxLrK1M/+KD2t6m1uiW5b6wjt4iI1kAY7P+sjH+rzgxtoZFbCqStH5Duk1b6gapYs7iggbbaHGcUUrdiEBw90o8a2z0Ybxn3T0M+tpQAX7gD4k765uDlNFlO+TmmU36M98K0TI88Yh+hxrp2ptyAAIzfQAAAA==', 'FedAuth': '77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAbGl2ZS5jb20sMCMuZnxtZW1iZXJzaGlwfGRlbmdqdW4ubGVpQGVyaWNzc29uLmNvbSwxMzI1ODQzNTkwMTAwMDAwMDAsMTMyNTIzODM2MDAwMDAwMDAwLDEzMjU4NTIyMzMwMjQ1OTU4MiwxMDEuMzMuMTI3LjExNSw2Niw5MmU4NGNlYi1mYmZkLTQ3YWItYmU1Mi0wODBjNmI4Nzk1M2YsLDNmYWUxYzM5LWEzOTktNDg0MC1iZmViLTc5NjgwMWJmNDczNCxmMzNhM2U0Ni1

In [6]:
print(ret.headers)

{'Cache-Control': 'private', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=utf-8', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'P3P': 'CP="ALL IND DSP COR ADM CONo CUR CUSo IVAo IVDo PSA PSD TAI TELo OUR SAMo CNT COM INT NAV ONL PHY PRE PUR UNI"', 'Set-Cookie': 'rtFa=OO+bIOYOBvyQa/pn+TjR0ere2fxDh7GzDRPBQ1f0cEcmOTJFODRDRUItRkJGRC00N0FCLUJFNTItMDgwQzZCODc5NTNGIzEzMjU4NDM1OTMwMjYxNjEzNiMzOTlEQUQ5Ri1BMDdELUIwMDAtQ0M4Ny1BRTkwMzI2Q0EzNzIBxW0xUovBgycOQ3YMd/qCMlEO80n0nd9Vhw23l6YobmRMWApdLwRz+lip86ew2dlMnXl53/RvYsEBm/tTLzTk649jJ6MywZFHnWGVmhFAsa9LT3PdeRhwqeohzWF1j+XqMng4sYg76Ub0QHJhTx23ywr2jrAwikeNrtizTLPS0CM4/qiVsZAnBhMORqb7fSxLrK1M/+KD2t6m1uiW5b6wjt4iI1kAY7P+sjH+rzgxtoZFbCqStH5Duk1b6gapYs7iggbbaHGcUUrdiEBw90o8a2z0Ybxn3T0M+tpQAX7gD4k765uDlNFlO+TmmU36M98K0TI88Yh+hxrp2ptyAAIzfQAAAA==; domain=sharepoint.com; path=/; SameSite=None; secure; HttpOnly, FedAuth=77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjgsMGguZnxtZW1iZXJzaGlwfDEwMDMyMDAwZTFkMjZjMGJAb

In [12]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_api/web/GetListUsingPath(DecodedUrl=@a1)/RenderListDataAsStream?@a1='/sites/PDURadioChengduFirmware/Shared Documents'&View=1d04f153-2aa3-4553-b6e2-14b66655e6d4&TryNewExperienceSingle=TRUE"
ret1=request.post(url,cookies=ret.cookies.get_dict())

In [15]:
import json
json.loads(ret1.content)

{'Row': [{'ID': '2875',
   'PermMask': '0x1b03c431aef',
   'FSObjType': '1',
   'HTML_x0020_File_x0020_Type': '',
   'UniqueId': '{562C9322-2B2D-4E69-9C12-4A642C4E2FB5}',
   'ProgId': '',
   'NoExecute': '0',
   'ContentTypeId': '0x0120007918351C8F30024CAE05B313360DF751',
   'FileRef': '/sites/PDURadioChengduFirmware/Shared Documents/2018 TeamBuilding',
   'FileRef.urlencode': '%2Fsites%2FPDURadioChengduFirmware%2FShared%20Documents%2F2018%20TeamBuilding',
   'FileRef.urlencodeasurl': '/sites/PDURadioChengduFirmware/Shared%20Documents/2018%20TeamBuilding',
   'FileRef.urlencoding': '/sites/PDURadioChengduFirmware/Shared%20Documents/2018%20TeamBuilding',
   'FileRef.scriptencodeasurl': '\\u002fsites\\u002fPDURadioChengduFirmware\\u002fShared Documents\\u002f2018 TeamBuilding',
   'SMTotalSize': '2285021874',
   'File_x0020_Size': '',
   'MediaServiceFastMetadata': '',
   '_CommentFlags': '',
   'File_x0020_Type': '',
   'HTML_x0020_File_x0020_Type.File_x0020_Type.mapall': 'icgen.gif|||'

In [16]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_api/web/GetListUsingPath(DecodedUrl=@a1)/RenderListDataAsStream?@a1='/sites/PDURadioChengduFirmware/Shared Documents'&RootFolder=/sites/PDURadioChengduFirmware/Shared Documents/2018 TeamBuilding&View=1d04f153-2aa3-4553-b6e2-14b66655e6d4&TryNewExperienceSingle=TRUE"
ret2=request.post(url,cookies=ret1.cookies.get_dict())

In [17]:
json.loads(ret2.content)

{'Row': [{'ID': '2934',
   'PermMask': '0x1b03c431aef',
   'FSObjType': '0',
   'HTML_x0020_File_x0020_Type': '',
   'UniqueId': '{9BDA7A81-1F41-4490-9839-9E1D50F4AD95}',
   'ProgId': '',
   'NoExecute': '1',
   'ContentTypeId': '0x01010007FC0BF75714AE4BAE808656DEB087A7',
   'FileRef': '/sites/PDURadioChengduFirmware/Shared Documents/2018 TeamBuilding/2018_TB_GroupPhotos.rar',
   'FileRef.urlencode': '%2Fsites%2FPDURadioChengduFirmware%2FShared%20Documents%2F2018%20TeamBuilding%2F2018%5FTB%5FGroupPhotos%2Erar',
   'FileRef.urlencodeasurl': '/sites/PDURadioChengduFirmware/Shared%20Documents/2018%20TeamBuilding/2018_TB_GroupPhotos.rar',
   'FileRef.urlencoding': '/sites/PDURadioChengduFirmware/Shared%20Documents/2018%20TeamBuilding/2018_TB_GroupPhotos.rar',
   'FileRef.scriptencodeasurl': '\\u002fsites\\u002fPDURadioChengduFirmware\\u002fShared Documents\\u002f2018 TeamBuilding\\u002f2018_TB_GroupPhotos.rar',
   'SMTotalSize': '311524490',
   'File_x0020_Size': '311468535',
   'MediaServ

In [9]:
url="https://ericsson.sharepoint.com/sites/PDURadioChengduFirmware/_layouts/15/download.aspx?UniqueId=0cf748e8%2D85b9%2D4b8a%2D9b9e%2Dda6ecf75d039"
ret3=request.post(url)

In [11]:
with open("testtest.docx", "wb") as f:
    f.write(ret3.content)

In [ ]:
from kgfw.extract.doc_handle import BaseDocxHandle

document = BaseDocxHandle(BytesIO(bytes(ret3.content)))
data = document.get_docx_structure()

In [ ]:
import json
import re
tmp = [
    {'id': 'a', 'name': 'hardware_environment1', 'status': 'free', 'is_data_updated': False},
    {'id': 'b', 'name': 'hardware_environment2', 'status': 'holding', 'is_data_updated': False},
    {'id': 'c', 'name': 'hardware_environment3', 'status': 'ready', 'is_data_updated': True}
]

In [ ]:
re.findall("\"name\": \"(.*?)\"",json.dumps(tmp)) 

In [1]:
import gzip
import requests
import json
import base64

In [2]:
content = requests.get('http://10.166.152.40/ru/docman/get')

In [3]:
test = eval(content.content.decode())['content']
temp = base64.b64decode(test[0])
temp = json.loads(gzip.decompress(temp).decode())

In [5]:
temp['children'] = ""

In [6]:
temp

{'id': 'a0fd644a734be4a00d86e9e917e3d51a',
 'name': 'Radon 1.1 SW IWD.docx',
 'children': '',
 'params': [{'name': 'address_map',
   'id': 'a0fd644a734be4a00d86e9e917e3d51a_address_map'}],
 'server': 'http://10.166.152.40/ru/docman/value'}

In [21]:
url = 'http://10.166.152.40/ru/docman/value'
data = {'sid':(None, 'ssh.root@100.98.146.93'),'docid':(None, 'a0fd644a734be4a00d86e9e917e3d51a'),'key':(None, 'MISC_HW_ID.ID,MISC_HW_REV.REV'),'addr_map':(None, 'x:0x1090000000 + (x&0xFFFFFFF)')}
r =requests.post(url,files=data)

In [3]:
import re

temp = '${bb}_${ee}_${interactive.layer}'
re.findall('\$\{(.*?)\}', temp)

['bb', 'ee', 'interactive.layer']